# Uses interaction to push triples to the brain and query it

Before running, start GraphDB and make sure that there is a sandbox repository.
GraphDB can be downloaded from:

https://graphdb.ontotext.com


In [1]:
import json
import os
import time
import uuid
from datetime import date
from datetime import datetime
from random import getrandbits, choice
import pathlib

# general imports for EMISSOR and the BRAIN
import emissor as em
import requests
from cltl import brain
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario

/Users/piek/PycharmProjects/cltl-chatbots/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Error loading punkt: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>


### Importing the chatbot utility functions

In [2]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util
import chatbots.util.capsule_util as c_util
import chatbots.intentions.talk as talk
import chatbots.intentions.get_to_know_you as friend

## Specifying the BRAIN

We imported the brain from CLTL. We create an instance using the class *LongTermMemory*, which takes three parameters: 1) the address of the triple store, 2) a path to a folder for logging the triples and a boolean flag for clearing the repository and reload the initial ontologies.

As the address of the triple store, we give the GraphDB localhost port (7200) and specify the name of the repository that we created beforehand. Any repository will do. We use here the *sandbox* repository. Note that you can also specify a remote SPARQL endpoint of another triple store or share a triple store among systems.

For storing the triples generate, we define a scenario folder in a data folder relative to where the notebooks are stored. It is based on the timestamp when we start. We will use this scenario structure later for the interaction as well as we did before.

Finall, if you set *clear_all* to *True*, the sandbox triple store is emptied (memory erased) and the basic ontological models are reloaded. Setting it to *False* means you add things to the current memory.

In [3]:
### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = os.path.abspath(os.path.join('../../data'))
if scenario_path not in sys.path:
    sys.path.append(scenario_path)

    ### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = os.path.abspath(os.path.join('../../data'))
if scenario_path not in sys.path:
    sys.path.append(scenario_path)

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)
    
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"
log_path = pathlib.Path(rdffolder)
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                                log_dir=log_path,
                                clear_all=True)


2021-11-10 14:03:06,916 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-11-10 14:03:08,690 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


### Posting triples to the brain

The *talk.py* script in *intentions* demonstrates the basic API functions for posting and querying the BRAIN. The input of for these function is a triple in JSON format. In the next cell, we define such a triple for the subject 'Fred, the predicate 'like' and the object 'singing'. In addition to a *label* that is used to identify the resource URI in the knowledge graph, we also need to provide type information.


In [4]:
test_triple = {'subject': {'label': 'Fred', 'type': ['noun.person']},
               'predicate': {'label': 'like', 'type': ['verb.emotion']},
               'object': {'label': 'singing', 'type': ['noun.act']}}

The BRAIN uses the roboGRaSP model to capture knowledge. In roboGRaSP, we keep track of the *mentioning* of knowledge in signals. This can be either through conversation or though perception. Whenever a signal makes reference to knowledge (e.g. people, objects, properties), there is also a source of the signal and possibly a perspective.
Sources are typically speakers with whom a system interacts but they can also be the camera or microphone of the agent that picked up a signal. Perspectives reflect the attitude and appraisal of the source towards the triple. Possible perspective values are, among others: certainty, sentiment, emotion, epistemic belief, deontic judgement, ethical judgment. Following GRaSP, the actual triple is reprsented as a claim made by the source in or through a signal and the perspective values are represented as attributions of the source.

In order to deal with these GRaSP layers, we need to embed the *factual* triple within a so-called *capsule* JSON structures that provides contextual information, the source and the perspective. The following attributes are required:

* The physical context is time and space: context_id, contry, region, city, place, place_id, position, date
* The physical objects in the physical context: objects, people
* The interactive context: chat, turn, utterance and utterance type 
* The author (source)
* The triple: subject, predicate and object
* The perspective

Next is a capsule example, that contains the above triple embedded within the minimal contextual information:

```
{'context_id': '1',
  'country': '',
  'city': '',
  'region': '',
  'place': '',
  'place_id': '',
  'position': '',
  'date': datetime.date(2021, 11, 9),
  'objects': [],
  'people': [],
  'author': 'me',
  'chat': '1',
  'turn': '1',
  'utterance': '',
  'utterance_type': <UtteranceType.STATEMENT: 0>
  'subject': {'label': 'Fred', 'type': ['agent']},
  'predicate': {'label': 'like', 'type': ['verb.emotion']},
  'object': {'label': 'Fred', 'type': ['noun.object']},
  'perspective': []
}
```

The *capsule_util.py* within util has a number of functions to create capsules. We use these functions within the different intentions that involve the brain, as shown below.

To post the above triple as a simple statement we use the *post_a_triple_and_get_thoughts* function from the *talk.py* that you find within the *intentions* module. This function takes a triple and an initialised brain as parameters and it returns the capsule that is created but also the response from the brain as a JSON structure. 

Posting triples to the brain is done through the *update* API function. This takes three parameters: the capsule and two boolean settings: *reason_types*  triggers linking subjects and entities to their types by consulting the semantic web, and *create_label* which triggers the system to create an additional *rdfs:label* property from the subject and object label if it is an entity. We typically set these to false, which are also the default values. Calling the update function not only stores the triple but is also followed by a series of preprogrammed SPAQRL queries that represent the *thoughts* on the changes to the brain.

In [6]:
import pprint


capsule, throughts_json = talk.post_a_triple_and_get_thoughts(test_triple, my_brain)
print("Capsule that provides the interactive contexts for the signal to which a triple is grouned:\n")
pprint.pprint(capsule)


2021-11-10 17:23:48,634 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: fred_like_singing [person_->_act])
2021-11-10 17:23:48,673 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Statement Novelty: 1 times, e.g. me on November,2021
Capsule that provides the interactive contexts for the signal to which a triple is grouned:

{'author': 'me',
 'chat': '1',
 'city': '',
 'context_id': '1',
 'country': '',
 'date': datetime.date(2021, 11, 10),
 'object': {'label': 'singing', 'type': ['noun.act']},
 'objects': [],
 'people': [],
 'perspective': <cltl.brain.infrastructure.api.Perspective object at 0x7fd28105bad0>,
 'place': '',
 'place_id': '',
 'position': '',
 'predicate': {'type': 'like'},
 'region': '',
 'subject': {'label': 'Fred', 'type': ['noun.person']},
 'triple': fred_like_singing [person_->_act]),
 'turn': '1',
 'type': <UtteranceType.STATEMENT: 0>,
 'utterance': '',
 'utterance_type': <UtteranceType.STATEMENT: 0>}


In the above print outs, we see the triple echoed in the capsule. After posting the triple, we could query the brain in GraphDB as is shown in the next image:

![graph-db query](images/graph-db-query.png "GraphDB query for the triple result")

We also caught the response of the brain to this new information, which looks as shown below. The *thoughts* element lists different possible issues: _complement_conflict,_complement_gaps, _entity_novelty, _negation_conflicts, _overlaps, _statement_novelty, _subject_gaps, trust. Since we have not added a lot of data, most elements are still empty.

In [8]:
print()
print("Thoughts resulting from posting the triples to the brain:\n")
pprint.pprint(throughts_json)


Thoughts resulting from posting the triples to the brain:

{'response': '204',
 'statement': {'author': 'me',
               'chat': '1',
               'city': '',
               'context_id': '1',
               'country': '',
               'date': '2021-11-10',
               'object': {'label': 'singing', 'type': ['noun.act']},
               'objects': [],
               'people': [],
               'perspective': {'_certainty': 'UNDERSPECIFIED',
                               '_emotion': 'UNDERSPECIFIED',
                               '_polarity': 'UNDERSPECIFIED',
                               '_sentiment': 'UNDERSPECIFIED',
                               '_time': None},
               'place': '',
               'place_id': '',
               'position': '',
               'predicate': {'type': 'like'},
               'region': '',
               'subject': {'label': 'Fred', 'type': ['noun.person']},
               'triple': {'_complement': {'_confidence': 0.0,
            

To verbalise the thoughts. we use a replier *LenkaReplier* imported from *cltl.reply_generation*. This replier randomly selects a thought and uses templates to generate a natural language phrase.

In [9]:
replier = LenkaReplier()
reply = replier.reply_to_statement(throughts_json, proactive=True, persist=True)
print(reply)


2021-11-10 17:46:56,850 -     INFO -   cltl.reply_generation.api.LenkaReplier - Booted
Interesting! I had never heard about singing before!


You can call the replier repititively for the same response and it may select a different thought or phrasing.

In [12]:
for i in range(10):
    reply = replier.reply_to_statement(throughts_json, proactive=True, persist=True)
    print(reply)

I just learned something, I had never heard about fred before!
If you don't mind me asking. What types can fred like
I just learned something, I had never heard about singing before!
I am curious. What types can fred like
I am curious. What types of act orInstance like singing do person orInstance usually like
Interesting! I had never heard about singing before!
I would like to know. What types of act orInstance like singing do person orInstance usually like
Interesting! I had never heard about fred before!
I have heard this before. me told me about it in 2021 11 10T00:00:00
Exciting news! I had never heard about singing before!


### Querying the brain

In [10]:
test_triple = {'predicate': {'label': 'label'}, 
               'subject': {'label': 'Fred', 'type': ['agent']}, 
               'object': {'label': 'Fred', 'type': ['noun.object']}}

thoughts = talk.post_a_triple(test_triple, replier, my_brain)
print(thoughts)

{'chat': '1', 'turn': '1', 'author': 'me', 'utterance': '', 'utterance_type': <UtteranceType.STATEMENT: 0>, 'position': '', 'subject': {'label': 'Fred', 'type': ['agent']}, 'predicate': {'type': 'label'}, 'object': {'label': 'Fred', 'type': ['noun.object']}, 'context_id': '1', 'date': datetime.date(2021, 11, 5), 'place': '', 'place_id': '', 'country': '', 'region': '', 'city': '', 'objects': [], 'people': []}
2021-11-05 14:54:44,452 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: fred_label_fred [agent_->_object])
2021-11-05 14:54:44,505 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: new subject - new object 
I did not know that! I had never heard about fred before!


In [12]:
test_triple = {'predicate': {'label': 'like', 'type': ['prep']}, 
               'subject': {'label': 'Fred', 'type': ['noun.person']}, 
               'object': {'label': 'singing', 'type': ['noun.act']}}

thoughts = talk.post_a_triple(test_triple, replier, my_brain)
print(thoughts)


{'chat': '1', 'turn': '1', 'author': 'me', 'utterance': '', 'utterance_type': <UtteranceType.STATEMENT: 0>, 'position': '', 'subject': {'label': 'Fred', 'type': ['noun.person']}, 'predicate': {'type': 'like'}, 'object': {'label': 'singing', 'type': ['noun.act']}, 'context_id': '1', 'date': datetime.date(2021, 11, 5), 'place': '', 'place_id': '', 'country': '', 'region': '', 'city': '', 'objects': [], 'people': []}
2021-11-05 14:55:14,251 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: fred_like_singing [person_->_act])
2021-11-05 14:55:14,295 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Statement Novelty: 1 times, e.g. me on November,2021
2021-11-05 14:55:14,401 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Overlaps: 0 subject overlaps: e.g. '' - 1 object overlaps: e.g. me on November,2021 about lenka
I am curious. What kinds of things can like a singing like fred


In [14]:
test_triple = {'predicate': {'label': 'like', 'type': ['verb.emotion']}, 
               'subject': {'label': 'fred', 'type': ['agent']}, 
               'object': {'label': '', 'type': []}}
answer = talk.answer_a_query(test_triple, replier, my_brain)
print(answer)

{'chat': '1', 'turn': '1', 'author': 'me', 'utterance': '', 'utterance_type': <UtteranceType.QUESTION: 1>, 'position': '', 'subject': {'label': 'fred', 'type': ['agent']}, 'predicate': {'type': 'like'}, 'object': {'label': '', 'type': []}, 'context_id': '1', 'date': datetime.date(2021, 11, 5), 'place': '', 'place_id': '', 'country': '', 'region': '', 'city': '', 'objects': [], 'people': []}
2021-11-05 14:55:43,863 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: fred_like_? [agent_->_])
I have no idea.


In [9]:
test_triple = {'predicate': {'label': 'like', 'type': ['verb.emotion']}, 
               'subject': {'label': 'fred', 'type': ['agent']}, 
               'object': {'label': '', 'type': []}}
answer = talk.answer_a_query(test_triple, replier, my_brain)
print(answer)

{'chat': '1', 'turn': '1', 'author': 'me', 'utterance': '', 'utterance_type': <UtteranceType.QUESTION: 1>, 'position': '', 'subject': {'label': 'fred', 'type': ['agent']}, 'predicate': {'type': 'like'}, 'object': {'label': '', 'type': []}, 'context_id': '1', 'date': datetime.date(2021, 11, 5), 'place': '', 'place_id': '', 'country': '', 'region': '', 'city': '', 'objects': [], 'people': []}
2021-11-05 14:53:02,874 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: fred_like_? [agent_->_])
I don't know


## Standard initialisation of a scenario

In [3]:
from random import getrandbits
import requests
##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = os.path.abspath(os.path.join('../../data'))
if scenario_path not in sys.path:
    sys.path.append(scenario_path)

    ### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = os.path.abspath(os.path.join('../../data'))
if scenario_path not in sys.path:
    sys.path.append(scenario_path)

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)

### Define the folder where the images and rdf are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"

### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario = scenarioStorage.create_scenario(scenario_id, datetime.now().microsecond, datetime.now().microsecond, AGENT)

Directory  ../../data/2021-11-04-17:39:09  Created 
Directory  ../../data/2021-11-04-17:39:09/image  Created 


In [7]:
#### Small sequence to learn name of speaker
initial_prompt = f"{choice(GREETING)} {choice(ASK_NAME)} {HUMAN_NAME}?"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

#### Get name from person 
HUMAN_NAME, HUMAN_ID = friend.get_a_name_and_id(scenario, AGENT)

chat = Chat(HUMAN_ID)

#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = f"{choice(TALK_TO_ME)}"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

utterance = ""
#### Get input and loop
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    ###### Getting the next input signals
    utterance = input('\n')
    print(HUMAN_NAME + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    #### Process input and generate reply
    reply = talk.process_text_and_reply_(test_triple, UtteranceType.QUESTION, scenario, place_id, location, HUMAN_ID, textSignal, chat, replier, my_brain)
    print(AGENT + ": " + reply)
    textSignal = d_util.create_text_signal(scenario, reply)
    scenario.append_signal(textSignal)


Leolani2: How's it going? What is your name? Piek?



 Piek


Piek
Leolani2: So your name is Piek?



 yes


2021-11-05 12:00:46,926 -     INFO - cltl.triple_extraction.api.Chat (Piek_t_926000)     000 - << Start of Chat with Piek_t_926000 >>
Leolani2: Would you like to chat? I'll do my best to keep up



 What is your name


Piek: What is your name
2021-11-05 12:01:34,222 -     INFO - cltl.triple_extraction.api.Chat (Piek_t_926000)     001 - Piek_t_926000: "What is your name"
UtteranceType.QUESTION
{'predicate': {'label': 'name-is', 'type': ['noun.communication', 'verb.stative']}, 'subject': {'label': 'leolani', 'type': ['agent']}, 'object': {'label': '', 'type': []}}
2021-11-05 12:01:34,228 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: leolani_like_? [agent_->_])
Leolani2: I wouldn't know!


KeyboardInterrupt: Interrupted by user

NameError: name 'answer_a_query' is not defined

In [8]:
scenario.scenario.ruler.end = datetime.now().microsecond
scenarioStorage.save_scenario(scenario)

## End of notebook